In [7]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.express as px
import wandb
import tensorflow as tf
import h5py
import numpy as np
import plotly.express as px
import flask
import scipy.stats
import plotly.graph_objects as go
import util
from modelzoo import *
import custom_fit

In [8]:
# y axis fixed somewhere in the Average
# remove mse
# dropdown menu for cell lines
# plot extra cell lines

CELL_LINE=1
server = flask.Flask(__name__)
app = dash.Dash(__name__, server=server)

In [9]:
def np_mse(a, b):
    return ((a - b)**2)

def scipy_pr(y_true, y_pred):

    pr = scipy.stats.pearsonr(y_true, y_pred)[0]
    return pr

def scipy_sc(a, b):
    sc = scipy.stats.spearmanr(a, b)
    return sc[0]

def np_poiss(y_true, y_pred):
    return y_pred - y_true * np.log(y_pred)

In [10]:
# def get_data(model_path, data_dir, fold_name='valid', N=10):
fold_name='valid'
N=10
model_path = '/home/shush/profile/QuantPred/wandb/run-20210623_073933-vwq5gdk5/files/best_model.h5'
#get data and model
trained_model = tf.keras.models.load_model(model_path, custom_objects={"GELU": GELU})
trained_model.compile(optimizer="Adam", loss='mse', metrics=['mse'])
foldset = util.make_dataset(data_dir, fold_name, util.load_stats(data_dir), coords=True, shuffle_data=False)

#get bin size
n_bins = trained_model.layers[-1].output_shape[1]
l_seq = trained_model.layers[0].input_shape[0][1]
bin_size = l_seq // n_bins

x_list = []
y_list = []
c_list = []
pred_list = []
for i, (coord, x, y) in enumerate(foldset):
    if i < N:
        x_cropped, y_cropped = custom_fit.center_crop(x,y,2048)
        y_binned = custom_fit.bin_resolution(y_cropped, bin_size)
        c_list += [str(c).strip('b\'chr').strip('\'') for c in coord.numpy()]
        x_list.append(x_cropped)
        y_list.append(y_binned[:,:,CELL_LINE])
        pred_list.append(trained_model.predict(x_cropped)[:,:,CELL_LINE])
    else:
        break



NameError: name 'data_dir' is not defined

In [ ]:
data_dict = {'y_true': np.concatenate(y_list), 'y_pred': np.concatenate(pred_list), 'coords': c_list}

In [ ]:
def create_scatter(data_dict):
    data = pd.DataFrame({'y_true': data_dict['y_true'].mean(axis=1),
                          'y_pred': data_dict['y_pred'].mean(axis=1),
                          'peak': np.array([data_dict['y_true'].max(axis=1)>=2][0], dtype=int)})
    fig = px.scatter(data, x='y_true',
                     y='y_pred', title='Average true vs pred', opacity=0.6,
                     color='peak', color_continuous_scale='viridis')
    fig.add_scatter(x=np.arange(0, 4), y=np.arange(0, 4), mode='lines', hoverinfo='skip', name='')
    return data, fig


In [6]:
data, fig = create_scatter(data_dict)

NameError: name 'create_scatter' is not defined

In [ ]:
@app.callback(
    dash.dependencies.Output('profile', 'figure'),
    [dash.dependencies.Input('predictions', 'hoverData')])
def update_profile(hoverData, data_dict=data_dict, data=data, bin_size=bin_size):
    x = hoverData['points'][0]['x']
    y = hoverData['points'][0]['y']
    seq_n = np.where(data==[x, y])[0][0]
    true_profile = np.repeat(data_dict['y_true'][seq_n,:], bin_size)
    pred_profile = np.repeat(data_dict['y_pred'][seq_n,:], bin_size)
    pr = scipy_pr(true_profile, pred_profile)
    fig = px.line(x=np.arange(len(true_profile)),
                  y = true_profile, title="Coverage, Pearson R={}, {}".format(np.around(pr,3),'chr'+data_dict['coords'][seq_n]))


    fig.add_scatter(x=np.arange(len(pred_profile)),
                    y = pred_profile,
                    name='Predicted', mode='lines', line=go.scatter.Line(color="red"))
    fig.update_layout()
    return fig

In [5]:
app.layout = html.Div([
    # main scatter plot panel
    html.Div([dcc.Graph(id='predictions', figure=fig)],
              style={'width': '49%', 'display': 'inline-block'}),
    html.Div([
              dcc.Graph(id='profile')],
              style={'width': '49%', 'display': 'inline-block'})
    #
    # ])

    ])

NameError: name 'fig' is not defined

In [100]:
app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


Traceback (most recent call last):
  File "/home/shush/tf_2/lib/python3.7/site-packages/flask/app.py", line 2088, in __call__
    return self.wsgi_app(environ, start_response)
  File "/home/shush/tf_2/lib/python3.7/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.handle_exception(e)
  File "/home/shush/tf_2/lib/python3.7/site-packages/flask/app.py", line 2070, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/shush/tf_2/lib/python3.7/site-packages/flask/app.py", line 1508, in full_dispatch_request
    self.try_trigger_before_first_request_functions()
  File "/home/shush/tf_2/lib/python3.7/site-packages/flask/app.py", line 1560, in try_trigger_before_first_request_functions
    self.ensure_sync(func)()
  File "/home/shush/tf_2/lib/python3.7/site-packages/dash/dash.py", line 1096, in _setup_server
    _validate.validate_layout(self.layout, self._layout_value())
  File "/home/shush/tf_2/lib/python3.7/site-packages/dash/_validate.py", line 364, in